In [1]:
import plotly.express as px
import numpy as np
import scipy as sp
import cellpose
import pandas as pd
import tifffile
import glob
from cellpose import models
import napari
%gui qt5
import skimage.measure as measure
import plotly.graph_objs as go
import scipy.stats as stats

In [2]:
viewer = napari.Viewer()

In [3]:
DAPI_channel = 0
piwi_channel = 1
mmp1_channel = 2
H3P_channel = 3

In [6]:
visualize = False

# Do cellpose to find nucleii, save

We will bin together 4 slices at a time, taking the mean for the fluorescence and taking the 2nd slice for nucleii segmentation

Only need to run this if have not already run it.

The tif files should be the raw files but with the background subtracted.

In [7]:
model_cyto = models.Cellpose(gpu=True, model_type='cyto')

In [8]:
fnames = glob.glob('./*/*.tif')

In [ ]:
for i,f in enumerate(fnames):
    print(i/len(fnames)*100)
    print(f)
    img = tifffile.imread(f)
    #img[:,0] = np.roll(img[:,0], 3, axis=0)
    #img[:,1] = np.roll(img[:,1], 2, axis=0)
    #img[:,2] = np.roll(img[:,2], 1, axis=0)
    if (len(img.shape)>4):
        img = img.mean(axis=0)
    
    new_frames = int(np.floor(img.shape[0]/6))
    
    reshaped_img = img[0:(new_frames*6),:,:,:].reshape([new_frames,6,4,2044,2048])

    flattened_img = np.mean(reshaped_img, axis=1)
    flattened_img[:,DAPI_channel,:,:] = np.max(reshaped_img[:,[2,3],DAPI_channel,:,:], axis=1)

    lst = [im for im in flattened_img[:,DAPI_channel,:,:]]
    nucleii = model_cyto.eval(lst, channels=[0,0], diameter=50, flow_threshold=0.7,
                           normalize=True, resample=False)[0]
    nucleii = np.array(nucleii)

    flattened_img = np.append(flattened_img, nucleii[:,np.newaxis,:], axis=1)
    tifffile.imwrite(f+'f', flattened_img.astype(np.uint16), imagej=True, metadata={'axes':'ZCYX'})

# Quantify the labeled files

In [9]:
fnames = glob.glob('./mmp1_piwi_h3p/*.tiff')

In [10]:
def get_intensities(imgs, labels):
    lst = []
    for im, lab in zip(imgs, labels):
        tlst = measure.regionprops(lab, im)
        [lst.append(t['intensity_mean']) for t in tlst]
    return np.array(lst)

In [11]:
def get_areas(labels):
    xylst = []
    zlst = []
    alst = []
    for slc, lab in enumerate(labels):
        tlst = measure.regionprops(lab, lab)
        [xylst.append(t['centroid']) for t in tlst]
        [alst.append(t['area']) for t in tlst]
        [zlst.append(slc) for t in tlst]
    return np.array(xylst), np.array(alst), np.array(zlst)

In [12]:
def filter_objects(mask, size):
    rtn = []
    for m in mask:
        labs = sp.ndimage.label(m)[0]
        props = measure.regionprops(labs, labs)
        lst = np.array([t['area'] for t in props])
        lst = np.append([0], lst)
        lst = lst>size
        size_img = lst[labs]
        rtn.append(size_img)
    return np.array(rtn)


In [13]:
def get_edt(mmp1, show=False):
    mmp1_smoothed = sp.ndimage.gaussian_filter(mmp1, sigma=[0.001,3,3])

    thresh = np.max([200,np.max(mmp1_smoothed)/200])
    print(thresh)
    mmp1_masked = mmp1_smoothed>thresh

    filtered = filter_objects(mmp1_masked,2000)

    edt = sp.ndimage.morphology.distance_transform_edt(filtered==0, sampling=[6,.16,.16] )
    if show:
        viewer.add_image(mmp1_masked, blending='additive')
        viewer.add_labels(filtered)
        viewer.add_image(edt, blending='additive')
    return edt

In [14]:
def get_distances(mmp1, labels, show=False):
    edt = get_edt(mmp1, show=show)
    return get_intensities(edt, labels)

In [15]:
def get_data(f, show=False):
    img = tifffile.imread(f)
    centroids, areas, slices = get_areas(img[:,4])
    c0 = get_intensities(img[:,0], img[:,4])
    c1 = get_intensities(img[:,1], img[:,4])
    c2 = get_intensities(img[:,2], img[:,4])
    c3 = get_intensities(img[:,3], img[:,4])
    if show:
        viewer.add_image(img, channel_axis=1)
    distances = get_distances(img[:,mmp1_channel], img[:,4], show=show)
    cdf = pd.DataFrame({'Area':areas, 'X':centroids[:,0], 'Y':centroids[:,1], 'Z':slices, 'C0':c0, 'C1':c1,
                       'C2':c2, 'C3':c3, 'Distance':distances})
    return cdf

In [22]:
df = pd.DataFrame()
for f in fnames:
    print(f)
    cdf = get_data(f)
    cdf['File'] = f
    df = pd.concat([df,cdf])

./mmp1_piwi_h3p\48hpa1_piwi488_mmp1568_h3p647_40x008.tiff
200.0


C:\Users\smc\AppData\Local\Temp\ipykernel_5228\2516766477.py:10: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  edt = sp.ndimage.morphology.distance_transform_edt(filtered==0, sampling=[6,.16,.16] )


./mmp1_piwi_h3p\48hpa2_piwi488_mmp1568_h3p647_40x009.tiff
200.0


C:\Users\smc\AppData\Local\Temp\ipykernel_5228\2516766477.py:10: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  edt = sp.ndimage.morphology.distance_transform_edt(filtered==0, sampling=[6,.16,.16] )


./mmp1_piwi_h3p\48hpa3_piwi488_mmp1568_h3p647_40x010.tiff
200.0


...

C:\Users\smc\AppData\Local\Temp\ipykernel_5228\2516766477.py:10: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  edt = sp.ndimage.morphology.distance_transform_edt(filtered==0, sampling=[6,.16,.16] )


./mmp1_piwi_h3p\intact4_piwi488_mmp1568_h3p647_40x001.tiff
200.0


C:\Users\smc\AppData\Local\Temp\ipykernel_5228\2516766477.py:10: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  edt = sp.ndimage.morphology.distance_transform_edt(filtered==0, sampling=[6,.16,.16] )


./mmp1_piwi_h3p\intact5_piwi488_mmp1568_h3p647_40x002.tiff
200.0


C:\Users\smc\AppData\Local\Temp\ipykernel_5228\2516766477.py:10: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  edt = sp.ndimage.morphology.distance_transform_edt(filtered==0, sampling=[6,.16,.16] )


In [23]:
df.to_csv('Quantification_mmp1.csv')

# Analyze results

In [16]:
df = pd.read_csv('Quantification_mmp1.csv')

In [17]:
df['Time'] = df['File'].str.split('\\').str[1].str.split('_').str[0].str[0:-2]

In [18]:
#df['Label'] = df['File'].str.split('_').str[0]
#df['Time'] = df['File'].str.split('_').str[3]
#df['ID'] = df['File'].str.split('_').str[4]
#df['Side'] = df['File'].str.split('_').str[5]

#df.loc[df['Label']=='No','Time'] = df.loc[df['Label']=='No','File'].str.split('_').str[2]
#df.loc[df['Label']=='No','ID'] = df.loc[df['Label']=='No','File'].str.split('_').str[3]
#df.loc[df['Label']=='No','Side'] = df.loc[df['Label']=='No','File'].str.split('_').str[4]



In [19]:
def ptile(arg):
    return np.percentile(arg, 30)

In [20]:
pt = df.pivot_table(index=['File','Z'], values=['C0','C1','C2', 'C3'], aggfunc=ptile)
df = df.merge(pt.reset_index().rename(columns={'C0':'C0ptile', 'C1':'C1ptile', 'C2':'C2ptile', 'C3':'C3ptile'}), on=['File','Z'])
df['C0raw'] = df['C0']
df['C1raw'] = df['C1']
df['C2raw'] = df['C2']
df['C0'] = df['C0']/df['C0ptile']
df['C1'] = df['C1']/df['C1ptile']
df['C2'] = df['C2']/df['C2ptile']
df['C3'] = df['C3']/df['C3ptile']

In [21]:
#pt = df.pivot_table(index=['Label', 'Time', 'ID','Z'], values=['C0','C1','C2', 'C3'], aggfunc=ptile)
#df = df.merge(pt.reset_index().rename(columns={'C0':'C0ptile', 'C1':'C1ptile', 'C2':'C2ptile', 'C3':'C3ptile'}), on=['Label',
#                                                                                                                     'Time', 'ID'
#                                                                                                                     ,'Z'])
#df['C0'] = df['C0']/df['C0ptile']
#df['C1'] = df['C1']/df['C1ptile']
#df['C2'] = df['C2']/df['C2ptile']
#df['C3'] = df['C3']/df['C3ptile']

In [22]:
df['L0'] = np.log(df['C0'])
df['L1'] = np.log(df['C1'])
df['L2'] = np.log(df['C2'])
df['L3'] = np.log(df['C3'])

In [23]:
tmp_df = df[((df['Z'] % 4 == 0) & (df['Z']<30) & (df['File']=='./mmp1_piwi_h3p\\6hpa3_piwi488_mmp1568_h3p647_40x005.tiff'))]
tmp_df['L1raw'] = np.log(tmp_df['C1raw']/ptile(tmp_df['C1raw']))
if visualize:
    px.histogram(tmp_df, title='Raw Log Piwi Intensity',
             x='L1raw', facet_row='Z',height=900, range_x=[-.6,2.5], range_y=[0,60])

C:\Users\smc\AppData\Local\Temp\ipykernel_2828\3313935479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['L1raw'] = np.log(tmp_df['C1raw']/ptile(tmp_df['C1raw']))


In [24]:
tmp_df = df[((df['Z'] % 4 == 0) & (df['Z']<30) & (df['File']=='./mmp1_piwi_h3p\\6hpa3_piwi488_mmp1568_h3p647_40x005.tiff'))]
tmp_df['L1raw'] = np.log(tmp_df['C1raw']/np.min(tmp_df['C1raw']))
if visualize:
    px.histogram(tmp_df, title='Corrected Log Piwi Intensity',
             x='L1', facet_row='Z',height=900, range_x=[-.6,2.5], range_y=[0,60])

C:\Users\smc\AppData\Local\Temp\ipykernel_2828\2661862960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['L1raw'] = np.log(tmp_df['C1raw']/np.min(tmp_df['C1raw']))


In [25]:
tmp_df = df[((df['Z']<15))]
if visualize:
    px.histogram(tmp_df, animation_frame='File',
             x='L1', facet_row='Z',height=1600)

In [26]:
if visualize:
    px.density_heatmap(df, x="L1", y="L3",  
                   nbinsx=50, nbinsy=50, height=1200, animation_frame='File',
                   color_continuous_scale=[(0, "blue"), (0.05, "green"), (0.4, 'yellow'), (1, "red")])
                  

In [15]:
#df['Label'] = df['File'].str.split('_').str[0]
#df['Time'] = df['File'].str.split('_').str[3]
#df['ID'] = df['File'].str.split('_').str[4]
#df['Side'] = df['File'].str.split('_').str[5]

#df.loc[df['Label']=='No','Time'] = df.loc[df['Label']=='No','File'].str.split('_').str[2]
#df.loc[df['Label']=='No','ID'] = df.loc[df['Label']=='No','File'].str.split('_').str[3]
#df.loc[df['Label']=='No','Side'] = df.loc[df['Label']=='No','File'].str.split('_').str[4]

#df.loc[df['Side']=='wound','Side'] = 'medial'

In [16]:
#orders = {'Side':['lateral', 'medial'], 'Time':['Intact', '6hpa', '48hpa']}

# Look at medial vs lateral and 6hpa vs 48hpa

### Threshold for piwi and markers

In [27]:
piwi_thresh = 1.2
#lineage_thresh = 1.4
H3p_thresh = 4.0

df = df[df['Z']<25]

In [28]:
df['H3pPos'] = df['L3']>H3p_thresh
df['PiwiPos'] = df['L1']>piwi_thresh
df['DoublePos'] = df['H3pPos'] & df['PiwiPos']

In [29]:
gdf = df.groupby(['Time', 'File']).agg({'H3pPos':np.mean, 'PiwiPos':np.mean, 'DoublePos':np.mean}).reset_index()
gdf['FractionDoublePos'] = gdf['DoublePos']/gdf['PiwiPos']

### Threshold based analysis

In [30]:

if visualize:
    f=go.FigureWidget(
        px.box(gdf, x='Time', y='PiwiPos', points='all',
            hover_data=['File'])
        )
     
    def click_fn(trace, points, state):
        
        if (len(points.point_inds)>0):
            file = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
            print(file)
            img = tifffile.imread(file)[0:25]
            viewer.layers.clear()
            viewer.add_image(img[:,0:4], channel_axis=1)
            viewer.add_labels(img[:,4])
            pts = (df[(df['File']==file) & (df['PiwiPos'])][['Z', 'X', 'Y']].to_numpy())
            viewer.add_points(pts, size=20, face_color='yellow')
            pts = (df[(df['File']==file) & (df['H3pPos'])][['Z', 'X', 'Y']].to_numpy())
            viewer.add_points(pts, size=20, face_color='cyan')
            viewer.layers[1].contrast_limits=[0,4000]
            #get_edt(img[0:25,0], show=True)
            #viewer.layers[0].colormap='green'
            
            #viewer.layers[0].visible=False
            viewer.layers[2].visible=False
            viewer.layers[4].visible=False
            viewer.layers[3].colormap='gray'
            viewer.layers[0].colormap='gray'
            viewer.layers[1].colormap='yellow'
            viewer.layers[2].colormap='magenta'
            viewer.layers[3].colormap='cyan'
            

    for a in range(0,len(f.data)):
        f.data[a].on_click(click_fn)

    f

## Minimum Distance analysis

In [31]:
if visualize:
    f = px.box(df[df['PiwiPos']], x='Time', y='Distance',  color='DoublePos',
          hover_data=['File'])
    f.write_html('MinDistByCell_mmp1.html')
    f

In [32]:
if visualize:
    f = px.violin(df[df['PiwiPos']], x='Time', y='Distance',  color='DoublePos',
          hover_data=['File'])
    f.write_html('MinDistByCell_mmp1.html')
    f

In [33]:
ddf = df[df['PiwiPos']].groupby(['File', 'DoublePos', 'Time']).agg({'Distance':np.median}).reset_index()

In [34]:
if visualize:
    f = px.box(ddf, x='Time', color='DoublePos', y='Distance', points='all', title='Minimum distance to mmp1 region, median over all piwi positive cells')
    f.write_html('MinDistByFile_mmp1.html')
    f

In [35]:
dddf = df.groupby(['File', 'DoublePos', 'Time']).agg({'Distance':np.median}).reset_index()
if visualize:
    px.box(dddf, x='Time', y='Distance', points='all', title='Cells minimum distance to mmp1 region, aggregated by image')

## Closeness analysis

In [36]:
df['Close'] = df['Distance']<20

In [29]:
#ttdf = df.copy()
#ttdf['Distance'] = ttdf['Distance']*1000000
df.to_csv('AllCell_mmp1.csv')

In [37]:
gdf = df.groupby(['Time', 'File', 'Close']).agg({'H3pPos':np.mean, 'PiwiPos':np.mean, 'DoublePos':np.mean}).reset_index()
gdf['FractionDoublePos'] = gdf['DoublePos']/gdf['PiwiPos']

### Looking at fraction of all cells, close vs far

In [38]:
if visualize:
    px.box(gdf, x='Time', y='PiwiPos', color='Close', points='all')

In [39]:
if visualize:
    px.box(gdf, x='Time', y='H3pPos', color='Close', points='all')

In [40]:
if visualize:
    px.box(gdf, x='Time', y='DoublePos', color='Close', points='all')

In [41]:
pdf = df[df['PiwiPos']].groupby(['Time', 'File', 'Close']).agg({'H3pPos':np.mean, 'PiwiPos':np.mean, 'DoublePos':np.mean}).reset_index()
pdf['FractionDoublePos'] = pdf['DoublePos']/pdf['PiwiPos']

In [42]:
if visualize:
    px.box(pdf, x='Time', y='DoublePos', color='Close', points='all')